## Make Database
### Includes code to:
### (1) Save any # of csv files of hot100 list data
### (2) Read one hot100 list into a pandas dataframe
### (3) Use the Track Name and Artist Name from that hot100 list to get lyrics from MXM 
>Includes fuzzy matching of artist name strings



In [33]:
import sys
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
print 'testing!'

testing!


In [3]:
# edit this cell with your API info for billboard and spotify

mxm_key = "f8a009fe01e53de1e8a1388e60812ef5"

In [26]:
import billboard
from billboard import ChartData
import pdb 
import numpy as np 
import pandas as pd

# for lyrics stuff (mxm)
from musixmatch.track import TracksCollection
from musixmatch.lyrics import Lyrics

# for fuzzy matching artist names (when matching hot100 song to lyrics in mxm)
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# for joining list of strings
import string

In [4]:
import os

In [5]:
date_in = '2016-07-09' #initialize
testname = './hot100/hot100.' + date_in + '.csv'
if os.path.exists(testname):
    print testname, ' exists'

./hot100/hot100.2016-07-09.csv  exists


In [35]:
## This cell saves individual csv files of the billboard hot100 list data
## It starts with the most current list and moves backward in time until num_lists are retrieved 

num_lists = 4000 #how many top100 lists do you want? (i.e. how many csv files do you want?)
                    # we'll break out of the loop when we have this many lists or when date_in = '08-09-1958'
chartname = 'hot-100'
#date_in = '2016-07-09' #initialize
date_in = '1958-08-23'
end_date = '1958-08-09'
list_count = 1
while list_count <= num_lists :
    print 'processing ', date_in
    fname = './hot100/hot100.' + date_in + '.csv' #name to save data to
    data = ChartData(chartname, date = date_in)
    previous = data.previousDate
    if not os.path.exists(fname):
            #special case for 1958-08-23
        if (date_in == '1958-08-23'):
            print 'manually fixing unicode error'
            for x in data:
                if x.title.split(' ')[0:3] == ['Nel','Blu','Dipinto']:
                    words_in_title = x.title.split(' ')
                    words_in_title.pop()
                    print 'Changing ', x.title, ' to ', string.join(words_in_title)
                    x.title = string.join(words_in_title)
        print fname, ' doesnt exist, so we will save this date...'
        print 'saving to : ', fname
        f = open(fname, 'wb')
        [f.write(x.spotifyID + '\t' + x.title + '\t'  + x.artist + '\t' + np.str(x.peakPos) + '\t' + 
            np.str(x.lastPos) + '\t' + np.str(x.weeks) + '\t' + np.str(x.rank) + '\t'  + np.str(x.change)
            + '\t' + x.spotifyLink + '\n')
            for x in data]
        f.close()
    list_count += 1
    if date_in == end_date:
        print 'done getting all the data'
        break
    date_in = previous

processing  1958-08-23
manually fixing unicode error
Changing  Nel Blu Dipinto Di Blu (VolarÃÂ©)  to  Nel Blu Dipinto Di Blu
./hot100/hot100.1958-08-23.csv  doesnt exist, so we will save this date...
saving to :  ./hot100/hot100.1958-08-23.csv
processing  1958-08-16
processing  1958-08-09
done getting all the data


Changing  Nel Blu Dipinto Di Blu (VolarÃÂ©)  to  Nel Blu Dipinto Di Blu


In [22]:
import string
liststr = ['Nel','Blu','Dipinto']
string.join(liststr)

'Nel Blu Dipinto'

In [ ]:
# Read in csv's into one large data frame:
f1 = 'hot100.2016-07-09.csv'
f2 = 'hot100.2016-07-09.csv'


In [45]:
# put one top100 list into a dataframe:
chartname = 'hot-100'
date_in = '2016-07-09' #initialize
print 'getting data for : ', date_in
data = ChartData(chartname, date = date_in)
spotid = []
titles = []
artists = []
peakpos = []
lastpos = []
weeks = []
rank = []
change = []
link = []
for x in data:
    spotid.append(x.spotifyID)
    titles.append(x.title)
    artists.append(x.artist)
    peakpos.append(x.peakPos)
    lastpos.append(x.lastPos)
    weeks.append(x.weeks)
    rank.append(x.rank)
    change.append(x.change)
    link.append(x.spotifyLink)

d = {'SpotifyID':spotid, 'Title':titles,'Artist':artists,'PeakPos':peakpos \
      ,'LastPos':lastpos,'Weeks':weeks,'Rank':rank,'Change':change,'SpotifyLink':link}
df = pd.DataFrame(data=d, index=spotid)
print 'Data Frame has ', len(df), ' songs in it'

getting data for :  2016-07-09
Data Frame has  100  songs in it


#### Cool, so we have the billboard data for one particular date in a DataFrame. 
#### Let's look at the top 5 lines:

In [46]:
df.head(5)

,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks
11hqMWwX7sF3sOGdtijofF,Drake Featuring WizKid & Kyla,0,1,1,1,11hqMWwX7sF3sOGdtijofF,https://embed.spotify.com/?uri=spotify:track:1...,One Dance,12
6JV2JOEocMgcZxYSZelKcc,Justin Timberlake,0,2,1,2,6JV2JOEocMgcZxYSZelKcc,https://embed.spotify.com/?uri=spotify:track:6...,Can't Stop The Feeling!,7
6r2jK1A6oFRPREZfxjc5d1,Desiigner,0,3,1,3,6r2jK1A6oFRPREZfxjc5d1,https://embed.spotify.com/?uri=spotify:track:6...,Panda,18
1i1fxkWeaMmKEB4T7zqbzK,The Chainsmokers Featuring Daya,0,4,3,4,1i1fxkWeaMmKEB4T7zqbzK,https://embed.spotify.com/?uri=spotify:track:1...,Don't Let Me Down,19
0azC730Exh71aQlOt9Zj3y,Calvin Harris Featuring Rihanna,+1,6,5,5,0azC730Exh71aQlOt9Zj3y,https://embed.spotify.com/?uri=spotify:track:0...,This Is What You Came For,8


In [47]:
# search spotify for the ID's listed in the one data frame we have...
spotid[0]

u'11hqMWwX7sF3sOGdtijofF'

#### We can query for just the first title in the data frame as follows

In [48]:
df['Title'][0]

u'One Dance'

### Let's just try to get lyrics for one song in the musicmatch database

### Use the first title in our Billboard DataFrame
#### How many different songs have this title? Who are the artists?

In [49]:
apikey = mxm_key

trackname_hot = df['Title'][0]
print 'trackname hot = ',trackname_hot
artist_hot = df['Artist'][0]
print 'artist hot = ', artist_hot
collection = TracksCollection.fromSearch(q_track=trackname_hot, apikey = apikey)
# print artists that have that title:
artists = [t['artist_name'] for t in collection]
print len(artists), ' songs in mxm have this title'
print ' artists : ', artists
# get the spotify ids too
sids = [t['track_spotify_id'] for t in collection]
print sids

trackname hot =  One Dance
artist hot =  Drake Featuring WizKid & Kyla
10  songs in mxm have this title
 artists :  [u'Conor Maynard', u'Audrey Hall', u'WattyJay', u'Alex Aiono', u'Drake feat. Wizkid and Kyla', u'Drake feat. Wizkid & Kyla', u'Paris Inc', u'Sara Farell', u'Boyce Avenue', u'vChenay']
[u'0ENy870oF03JUjU41Zf9iP', u'0hlnqmmraol40xRgkeqDhU', u'2Mn7ee2bQN4ysUac4rMtsa', u'54nZR2nv5hmhHI2YM0LOCE', u'', u'', u'5L860D530vnVdHWCjIXSdW', u'235dYhgn9D1iLqwMmf29Zd', u'7vOfJJRgEM4aelrcFmiEQv', u'42w2ZBO8UHfwoYuJ26XVYq']


### So there are 10 matching songs. We can see that 2 of these should give us the right lyrics (Drake)

### Note : These 2 matching songs don't have Spotify ID's, suggesting it may be best to match on Song name and Artist name rather than SID

In [50]:
# get the track id's for these 10 songs
track_id = [t['track_id'] for t in collection]

# select the first song with first word = "Drake"
i = 0
for a in artists:
    firstword = a.split(' ')[0]
    if firstword == 'Drake':
        track_id_to_select = track_id[i]
        break
    else:
        i += 1
        continue
print 'We matched hot 100 artist : ', artist_hot, ' with mxm artist : ', artists[i]


We matched hot 100 artist :  Drake Featuring WizKid & Kyla  with mxm artist :  Drake feat. Wizkid and Kyla


In [51]:
# Now let's display the lyrics for this song:

lyrics = Lyrics(track_id = track_id_to_select, apikey = apikey)
#print the lyrics body
print lyrics['lyrics_body']
#save the headers of lyrics object for later when we get lots of data:
lyric_df_cols = [str(x) for x in lyrics.keys()]
print 'keys in lyrics object: ', lyric_df_cols

Baby I like your style

Grips on your waist
Front way, back way
You know that I don't play
Streets not safe
But I never run away
Even when I'm away
Oti, oti, there's never much love when we go OT
I pray to make it back in one piece
I pray, I pray

That's why I need a one dance
Got a Hennessy in my hand
One more time 'fore I go
Higher powers taking a hold on me
I need a one dance
Got a Hennessy in my hand
One more time 'fore I go
Higher powers taking a hold on me

Baby I like your style

Strength and guidance
All that I'm wishing for my friends
...

******* This Lyrics is NOT for Commercial use *******
keys in lyrics object:  ['lyrics_copyright', 'lyrics_body', 'publisher_list', 'can_edit', 'writer_list', 'instrumental', 'verified', 'html_tracking_url', 'lyrics_language_description', 'locked', 'restricted', 'explicit', 'updated_time', 'lyrics_id', 'script_tracking_url', 'action_requested', 'pixel_tracking_url', 'lyrics_language']


#### Matching only the first word of each artist name won't always work, and neither will matching the entire string
> For example see the code below where 2 strings a human would recognize as meaning the same thing aren't considered ideintical by the computer for obvious reasons:

In [52]:
# Often the artist names won't match character for character, for example:
is_true = [artists[7] == artist_hot]
print 'Does ', artists[i], ' == ', artist_hot, ' ? '
print is_true


Does  Drake feat. Wizkid and Kyla  ==  Drake Featuring WizKid & Kyla  ? 
[False]


### One quick way to find fuzzy matches for the artist names is to use the fuzzywuzzy package. Below I've written a function that takes a list of artist names and the name we're trying to match as input, and outputs the index of the best match in the list. 

In [53]:
# make function that takes a list of artists, and 
def find_artist_match(name_to_match, artist_list):
    index = 0 #index of the artist we're scoring
    r = 0 #here i'll store the highest similarity score
    for a in artist_list:
        rnew = fuzz.ratio(name_to_match, a)
        print 'similarity between ', name_to_match, ' and ', a, ' : ', rnew
        if rnew > r: 
            best_match = a
            best_ind = index
            r = rnew
        index = index+1

    print 'Best Match = ', best_match
    assert(best_match==artist_list[best_ind])
    print 'Score = ',r
    return best_ind

#test
ind = find_artist_match(artist_hot, artists)
print artists[ind]

similarity between  Drake Featuring WizKid & Kyla  and  Conor Maynard  :  24
similarity between  Drake Featuring WizKid & Kyla  and  Audrey Hall  :  25
similarity between  Drake Featuring WizKid & Kyla  and  WattyJay  :  22
similarity between  Drake Featuring WizKid & Kyla  and  Alex Aiono  :  21
similarity between  Drake Featuring WizKid & Kyla  and  Drake feat. Wizkid and Kyla  :  75
similarity between  Drake Featuring WizKid & Kyla  and  Drake feat. Wizkid & Kyla  :  81
similarity between  Drake Featuring WizKid & Kyla  and  Paris Inc  :  21
similarity between  Drake Featuring WizKid & Kyla  and  Sara Farell  :  35
similarity between  Drake Featuring WizKid & Kyla  and  Boyce Avenue  :  20
similarity between  Drake Featuring WizKid & Kyla  and  vChenay  :  17
Best Match =  Drake feat. Wizkid & Kyla
Score =  81
Drake feat. Wizkid & Kyla


In [55]:
num_songs = 3 #how many songs to get from the hot100 song list

df_mxm_info = pd.DataFrame() #dataframe for some song info (from collections object)
#df_mxm_lyrics = pd.DataFrame() #dataframe for lyrics and related attributes
lyrics_list = [] #append lyric info to a list for each song, later we'll put it in a dataframe!


#header for info to be stored in df_mxm_info
cols = ['lyrics_id', 'has_lyrics', 'artist_name', 'track_spotify_id','track_id','track_name']

#header for info in df_mxm_lyrics
lyrics_df_cols = ['lyrics_copyright', 'lyrics_body','publisher_list', 'can_edit', 'writer_list', \
                  'instrumental','verified','html_tracking_url','lyrics_language_description', \
                  'locked','restricted','explicit','updated_time','lyrics_id','script_tracking_url', \
                  'action_requested','pixel_tracking_url','lyrics_language']

for i in xrange(num_songs):
    trackname_hot = df['Title'][i]
    print 'trackname hot = ',trackname_hot
    artist_hot = df['Artist'][i]
    print 'artist hot = ', artist_hot

    collection = TracksCollection.fromSearch(q_track=trackname_hot, apikey = apikey)

    # print artists that have that title:
    artists = [t['artist_name'] for t in collection]
    print 'There are ', len(artists), ' artists with this song title'
    print artists
    track_id = [t['track_id'] for t in collection]

    # fuzzy match the artist from hot100 to all artists matching track name
    ind = find_artist_match(artist_hot, artists)
    print artists[ind]

    # select the best track id (best match to artist name)
    track_id_selected = track_id[ind]
    print track_id_selected
    
    # store only one row in df_mxm_info
    # this row represents the best match to the billboard title+artist
    subset = pd.DataFrame.from_dict(collection)[cols] #could set index here to track_id or lyrics_id...
    is_true = subset['track_id'] == track_id_selected
    df_mxm_info = df_mxm_info.append(subset[is_true])
    
    # now add to the lyrics data frame as well:
    lyrics = Lyrics(track_id = track_id_selected, apikey = apikey)
    
    #df_mxm_lyrics = df_mxm_lyrics.append(lyrics.values())
    lyrics_list.append(lyrics.values())

trackname hot =  One Dance
artist hot =  Drake Featuring WizKid & Kyla
There are  10  artists with this song title
[u'Conor Maynard', u'Audrey Hall', u'WattyJay', u'Alex Aiono', u'Drake feat. Wizkid and Kyla', u'Drake feat. Wizkid & Kyla', u'Paris Inc', u'Sara Farell', u'Boyce Avenue', u'vChenay']
similarity between  Drake Featuring WizKid & Kyla  and  Conor Maynard  :  24
similarity between  Drake Featuring WizKid & Kyla  and  Audrey Hall  :  25
similarity between  Drake Featuring WizKid & Kyla  and  WattyJay  :  22
similarity between  Drake Featuring WizKid & Kyla  and  Alex Aiono  :  21
similarity between  Drake Featuring WizKid & Kyla  and  Drake feat. Wizkid and Kyla  :  75
similarity between  Drake Featuring WizKid & Kyla  and  Drake feat. Wizkid & Kyla  :  81
similarity between  Drake Featuring WizKid & Kyla  and  Paris Inc  :  21
similarity between  Drake Featuring WizKid & Kyla  and  Sara Farell  :  35
similarity between  Drake Featuring WizKid & Kyla  and  Boyce Avenue  :  20

In [56]:
df_mxm_info

,lyrics_id,has_lyrics,artist_name,track_spotify_id,track_id,track_name
5,15092857,1,Drake feat. Wizkid & Kyla,,108888372,One Dance
4,15311374,1,Justin Timberlake,,110789083,Can't Stop the Feeling
0,15300601,1,Desiigner,5OOkp4U9P9oL23maHFHL1h,113492519,Panda


In [57]:
#check that lyrics list is same length as info df
len(lyrics_list) == len(df_mxm_info)

True

In [58]:
# Store the lyrics list elements in a data frame
df_mxm_lyrics = pd.DataFrame(data= lyrics_list, columns=lyrics_df_cols) 
df_mxm_lyrics

,lyrics_copyright,lyrics_body,publisher_list,can_edit,writer_list,instrumental,verified,html_tracking_url,lyrics_language_description,locked,restricted,explicit,updated_time,lyrics_id,script_tracking_url,action_requested,pixel_tracking_url,lyrics_language
0,Lyrics powered by www.musiXmatch.com. This Lyr...,Baby I like your style\n\nGrips on your waist\...,[],0,[],0,0,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,1,0,0,2016-05-13T11:00:53Z,15092857,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en
1,Lyrics powered by www.musiXmatch.com. This Lyr...,I got this feeling inside my bones\nIt goes el...,[],0,[],0,0,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,1,0,0,2016-05-05T22:44:34Z,15311374,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en
2,Lyrics powered by www.musiXmatch.com. This Lyr...,This what they all been waitin' for\nI guess s...,[],0,[],0,0,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,1,0,1,2016-05-05T15:33:36Z,15300601,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en


In [59]:
# merge these 2 dataframes
df_mxm = pd.merge(df_mxm_info,df_mxm_lyrics,on='lyrics_id')

In [60]:
df_mxm.head()

,lyrics_id,has_lyrics,artist_name,track_spotify_id,track_id,track_name,lyrics_copyright,lyrics_body,publisher_list,can_edit,...,html_tracking_url,lyrics_language_description,locked,restricted,explicit,updated_time,script_tracking_url,action_requested,pixel_tracking_url,lyrics_language
0,15092857,1,Drake feat. Wizkid & Kyla,,108888372,One Dance,Lyrics powered by www.musiXmatch.com. This Lyr...,Baby I like your style\n\nGrips on your waist\...,[],0,...,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,1,0,0,2016-05-13T11:00:53Z,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en
1,15311374,1,Justin Timberlake,,110789083,Can't Stop the Feeling,Lyrics powered by www.musiXmatch.com. This Lyr...,I got this feeling inside my bones\nIt goes el...,[],0,...,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,1,0,0,2016-05-05T22:44:34Z,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en
2,15300601,1,Desiigner,5OOkp4U9P9oL23maHFHL1h,113492519,Panda,Lyrics powered by www.musiXmatch.com. This Lyr...,This what they all been waitin' for\nI guess s...,[],0,...,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,1,0,1,2016-05-05T15:33:36Z,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en


### To Do : 
#### Scale up!
#### Make the match more efficient and less error-prone (e.g., first look for spotify id match, then do the fuzzy join. also, check that has_lyrics is true before selecting?)
#### Eventually, join with EchoNest features